# 7-DOF Robot Topology Optimization using Pinocchio + CasADi

This notebook demonstrates how to optimize the link lengths of a 7-DOF robotic arm using CasADi and Pinocchio.  
The objective is to maximize the average manipulability over a set of sampled configurations.


In [47]:
import numpy as np
from casadi import SX, vertcat, Function, sum1, nlpsol
from pinocchio import casadi as cpin
import pinocchio as pin
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf
import time
import scipy.linalg
from scipy.spatial import ConvexHull
from pinocchio.visualize import MeshcatVisualizer
from IPython.display import display
from hppfcl import Box


## Define link length variables
We define symbolic variables for the 7 link lengths and axes order for each links.


In [48]:
# --- Step 1: Define symbolic variables
L = SX.sym("L", 7)
Y = SX.sym("Y", 7)
# theta = SX.sym("theta", 21)
q = SX.sym("q", 7)

# --- Step 2: Define joint axes (symbolic)
joint_axes = [
    SX([1.0, 0.0, 0.0]),
    SX([0.0, 1.0, 0.0]),
    SX([0.0, 0.0, 1.0]),
    SX([1.0, 0.0, 0.0]),
    SX([0.0, 0.0, 1.0]),
    SX([1.0, 0.0, 0.0]),
    SX([0.0, 0.0, 1.0])
]

## Build a casadi 7-DOF planar arm in Pinocchio



In [49]:
def build_f_kine_fixed():
    model = cpin.Model()
    joint_placement = cpin.SE3.Identity()
    parent_id = 0

    for i in range(7):
        axis_i = joint_axes[i]
        joint_id = model.addJoint(
            parent_id,
            cpin.JointModelRevoluteUnaligned(axis_i),
            joint_placement,
            f"joint{i}"
        )
        model.appendBodyToJoint(joint_id, cpin.Inertia.Random(), cpin.SE3.Identity())
        model.addFrame(cpin.Frame(f"link{i}", joint_id, 0, cpin.SE3.Identity(), cpin.FrameType.BODY))

        # Offset from current joint to next joint using homogeneous transform
        R = SX.eye(3)
        p = vertcat(L[i], Y[i], 0)
        T_offset = SX.zeros(4, 4)
        T_offset[:3, :3] = R
        T_offset[:3, 3] = p
        T_offset[3, 3] = 1.0
        offset = cpin.SE3(T_offset)

        joint_placement = joint_placement * offset
        parent_id = joint_id

    # Add hand effector frame at the tip of the last link
    T_eff = SX.zeros(4, 4)
    T_eff[:3, :3] = SX.eye(3)
    T_eff[3, 3] = 1.0
    T_effector = cpin.SE3(T_eff)
    model.addFrame(cpin.Frame("hand_effector", parent_id, 0, T_effector, cpin.FrameType.OP_FRAME))

    data = cpin.Data(model)
    cpin.forwardKinematics(model, data, q)
    cpin.updateFramePlacements(model, data)
    p_ee = data.oMf[model.getFrameId("hand_effector")].translation

    return Function("f_kine_fixed", [L, Y, q], [p_ee])

f_kine = build_f_kine_fixed()

##  Manipulability score function
This function computes the average Yoshikawa manipulability index over a set of random joint configurations.


In [50]:
# --- Step 4: Define workspace metric function
def build_workspace_metric(N=600):
    q_lower = np.deg2rad([-170, -15, -170, -120, -170, -30, -45])
    q_upper = np.deg2rad([170, 180, 170, 120, 170, 60, 80])
    q_samples = np.random.uniform(q_lower, q_upper, size=(N, 7))
    p_samples = [f_kine(L, Y, q_sample) for q_sample in q_samples]
    mean_p = sum(p_samples) / N
    spread = sum([sum1((p - mean_p)**2) for p in p_samples]) / N
    return Function("workspace_metric", [L, Y], [spread]), q_samples

Yoshikawa

In [51]:
def build_manipulability_metric(N=600):
    from casadi import trace
    q_lower = np.deg2rad([-170, -15, -170, -120, -170, -30, -45])
    q_upper = np.deg2rad([170, 180, 170, 120, 170, 60, 80])
    q_samples = np.random.uniform(q_lower, q_upper, size=(N, 7))

    # Create symbolic Jacobian function
    def build_f_jacobian():
        model = cpin.Model()
        joint_placement = cpin.SE3.Identity()
        parent_id = 0
        for i in range(7):
            axis_i = joint_axes[i]
            joint_id = model.addJoint(
                parent_id,
                cpin.JointModelRevoluteUnaligned(axis_i),
                joint_placement,
                f"joint{i}"
            )
            model.appendBodyToJoint(joint_id, cpin.Inertia.Random(), cpin.SE3.Identity())
            model.addFrame(cpin.Frame(f"link{i}", joint_id, 0, cpin.SE3.Identity(), cpin.FrameType.BODY))

            T_offset = cpin.SE3(SX.eye(3), vertcat(L[i], Y[i], 0))
            joint_placement = joint_placement * T_offset
            parent_id = joint_id

        T_effector = cpin.SE3(SX.eye(3), SX([0.0, 0.0, 0.0]))
        model.addFrame(cpin.Frame("hand_effector", parent_id, 0, T_effector, cpin.FrameType.OP_FRAME))

        data = cpin.Data(model)
        cpin.forwardKinematics(model, data, q)
        cpin.updateFramePlacements(model, data)
        frame_id = model.getFrameId("hand_effector")
        J = cpin.computeFrameJacobian(model, data, q, frame_id, cpin.ReferenceFrame.LOCAL_WORLD_ALIGNED)
        return Function("f_jacobian", [L, Y, q], [J])

    f_jacobian = build_f_jacobian()
    w_samples = []
    for q_sample in q_samples:
        J = f_jacobian(L, Y, q_sample)
        JJt = J[:3, :] @ J[:3, :].T
        manipulability = trace(JJt)  # proxy for isotropic spread
        w_samples.append(manipulability)

    manipulability_score = sum(w_samples) / N
    return Function("workspace_metric", [L, Y], [manipulability_score]), q_samples

In [52]:
workspace_metric, q_samples = build_manipulability_metric()

## Optimization with CasADi
We define an objective function that returns the negative manipulability score for given link lengths, and solve it using IPOPT.


In [53]:
# --- Step 5: Optimize workspace metric
x = vertcat(L, Y)  # seulement L et Y
nlp = {"x": x, "f": -workspace_metric(x[:7], x[7:])}
solver = nlpsol("solver", "ipopt", nlp)

x0 = np.concatenate([np.ones(7) * 0.3, np.zeros(7)])
lbx = [0.05]*7 + [-0]*7
ubx = [0.15]*7 + [0]*7

result = solver(x0=x0, lbx=lbx, ubx=ubx)
x_opt = result["x"].full().flatten()

L_opt = x_opt[:7]
Y_opt = x_opt[7:14]

print("\n✅ Optimized link lengths:", L_opt)
print("✅ Optimized Y offsets:", Y_opt)



This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        7
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -1.2385141e+01 0.00e+00 3.45e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

##  Build the robot
We build the robot using the optimal link lengths


In [54]:
# --- Step 6: Build real Pinocchio model for Meshcat viewer

def build_pin_model(L_vals, Y_vals):
    model = pin.Model()
    visuals = pin.GeometryModel()
    joint_placement = pin.SE3.Identity()
    parent_id = 0

    axes_list = [
        np.array([1.0, 0.0, 0.0]),  # shoulder roll
        np.array([0.0, 1.0, 0.0]),  # shoulder pitch
        np.array([0.0, 0.0, 1.0]),  # shoulder yaw
        np.array([1.0, 0.0, 0.0]),  # elbow roll
        np.array([0.0, 0.0, 1.0]),  # elbow yaw
        np.array([1.0, 0.0, 0.0]),  # wrist roll
        np.array([0.0, 0.0, 1.0])   # wrist yaw
    ]

    for i in range(7):
        axis = axes_list[i]
        joint_id = model.addJoint(
            parent_id,
            pin.JointModelRevoluteUnaligned(axis),
            joint_placement,
            f"joint{i}"
        )
        inertia = pin.Inertia.Random()
        model.appendBodyToJoint(joint_id, inertia, pin.SE3.Identity())
        model.addFrame(pin.Frame(f"link{i}", joint_id, 0, pin.SE3.Identity(), pin.FrameType.BODY))

        # Offset transform for next joint relative to this joint
        T_offset = pin.SE3(np.eye(3), np.array([float(L_vals[i]), float(Y_vals[i]), 0.0]))
        joint_placement = joint_placement * T_offset  


        # Visualisation
        color = np.random.rand(4)
        color[3] = 0.8

        # Position du visuel au milieu du link, dans le repère local
        box_x = Box(float(L_vals[i]), 0.02, 0.02)
        offset_x = pin.SE3(np.eye(3), np.array([float(L_vals[i]) / 2, 0.0, 0.0]))
        geom_obj_x = pin.GeometryObject(f"visual_joint{i}_x", joint_id, joint_id, offset_x, box_x)

        geom_obj_x.meshColor = color
        visuals.addGeometryObject(geom_obj_x)

        if abs(Y_vals[i]) > 1e-6:
            box_y = Box(0.02, float(abs(Y_vals[i])), 0.02)
            offset_y = pin.SE3(np.eye(3), np.array([0.0, Y_vals[i] / 2.0, 0.0]))
            geom_obj_y = pin.GeometryObject(f"visual_joint{i}_y", joint_id, joint_id, offset_y, box_y)

            visuals.addGeometryObject(geom_obj_y)

        parent_id = joint_id

    # Add hand effector frame at the end of the last link
    T_effector = pin.SE3(np.eye(3), np.array([0.0, 0.0, 0.0]))
    model.addFrame(pin.Frame("hand_effector", parent_id, 0, T_effector, pin.FrameType.OP_FRAME))

    return model, visuals

pin_model, visual_model = build_pin_model(L_opt, Y_opt)
pin_data = pin_model.createData()

##  Visualize the optimized robot
We display it using Meshcat.


In [55]:
viz = MeshcatVisualizer(pin_model, visual_model, visual_model)
viz.initViewer()
viz.loadViewerModel()
viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/


In [56]:
# --- Step 9: Display workspace points and convex hull
workspace_points = []
model = pin_model
data = model.createData()
endEffector_ID = model.getFrameId("hand_effector")

# for i, q_sample in enumerate(q_samples):
#     pin.forwardKinematics(model, data, q_sample)
#     pin.updateFramePlacements(model, data)
#     p = data.oMf[endEffector_ID].translation
#     workspace_points.append(p)
#     sphere = g.Sphere(0.01)
#     mat = g.MeshLambertMaterial(color=0xff0000)
#     viz.viewer[f"workspace/pt_{i}"].set_object(sphere, mat)
#     viz.viewer[f"workspace/pt_{i}"].set_transform(tf.translation_matrix(p))

try:
    workspace_points = np.array(workspace_points)
    hull = ConvexHull(workspace_points)
    viz.viewer["workspace/hull"].set_object(
        g.TriangularMeshGeometry(workspace_points, hull.simplices),
        g.MeshLambertMaterial(color=0x00ff00, opacity=0.3, transparent=True)
    )
    print("✅ Convex hull volume:", hull.volume)
except Exception as e:
    print("⚠️ Convex hull failed:", e)

def displayScene(q, dt):
    pin.forwardKinematics(model, data, q)
    pin.updateFramePlacements(model, data)

    M = data.oMf[endEffector_ID]

    # Afficher une sphère au niveau du hand effector
    sphere = g.Sphere(0.02)
    mat = g.MeshLambertMaterial(color=0xffff00)  # jaune
    viz.viewer["hand_effector/sphere"].set_object(sphere, mat)
    viz.viewer["hand_effector/sphere"].set_transform(tf.translation_matrix(M.translation))

    viz.display(q)
    time.sleep(dt)

def displayTraj(qs, dt):
    for q in qs:
        displayScene(q, dt)

# Generate and animate trajectory
n_steps = 100
q_traj = np.linspace(-np.pi / 4, np.pi / 4, n_steps).reshape(-1, 1) * np.ones((n_steps, model.nq))

print("\n🎞️ Displaying animated trajectory...")
displayTraj(q_traj, dt=0.05)

⚠️ Convex hull failed: tuple index out of range

🎞️ Displaying animated trajectory...


In [57]:
displayScene(np.zeros(7), dt=0.05)  # Display last configuration